In [ ]:
!pip install gnews

In [6]:
from bs4 import BeautifulSoup
import json
import numpy as np
import requests
from requests.models import MissingSchema
import trafilatura
from gnews import GNews
import pandas as pd
import re
import requests
from bs4 import BeautifulSoup
import json
import numpy as np
import trafilatura

08/31/2023 06:09:27 PM - NumExpr defaulting to 8 threads.


In [7]:
google_news = GNews()
google_news.period = '7d'  # News from last 7 days
google_news.max_results = 150  # number of responses across a keyword
google_news.country = 'United States'  # News from a specific country 
google_news.language = 'english'  # News in a specific language
#google_news.exclude_websites = ['yahoo.com', 'cnn.com']  # Exclude news from specific website i.e Yahoo.com and CNN.com
#google_news.start_date = (2020, 1, 1) # Search from 1st Jan 2020
#google_news.end_date = (2020, 3, 1) # Search until 1st March 2020

In [8]:
AAPL_news = google_news.get_news('AAPL')

In [9]:
# Extract relevant fields
df_data = []
for entry in AAPL_news:
    title = entry['title']
    url = entry['url']
    published_date = entry['published date']
    publisher = entry['publisher']['title']
    df_data.append({'title': title, 'url': url, 'published_date': published_date, 'publisher': publisher})

# Create DataFrame
df = pd.DataFrame(df_data)

# Print the DataFrame
df

,title,url,published_date,publisher
0,"Apple (NASDAQ:AAPL) Gains Slightly, Though Har...",https://news.google.com/rss/articles/CBMiWGh0d...,"Wed, 30 Aug 2023 16:29:57 GMT",TipRanks
1,Unusual Put Option Trade in Apple (AAPL) Worth...,https://news.google.com/rss/articles/CBMiVmh0d...,"Tue, 29 Aug 2023 15:27:00 GMT",Nasdaq
2,Apple Inc (AAPL) Stock Gains 1.82% This Week: ...,https://news.google.com/rss/articles/CBMib2h0d...,"Thu, 24 Aug 2023 19:01:41 GMT",InvestorsObserver
3,"After Hours Most Active for Aug 25, 2023 : AAP...",https://news.google.com/rss/articles/CBMiemh0d...,"Fri, 25 Aug 2023 20:29:00 GMT",Nasdaq
4,"After Hours Most Active for Aug 28, 2023 : IGS...",https://news.google.com/rss/articles/CBMieWh0d...,"Mon, 28 Aug 2023 20:29:00 GMT",Nasdaq
...,...,...,...,...
87,"Dow, Nasdaq Shed Triple Digits; VIX Snaps Losi...",https://news.google.com/rss/articles/CBMiVWh0d...,"Thu, 24 Aug 2023 20:27:00 GMT",Nasdaq
88,Warren Buffett Will Collect $4.31 Billion in A...,https://news.google.com/rss/articles/CBMid2h0d...,"Fri, 25 Aug 2023 09:06:00 GMT",Nasdaq
89,SPLG: The Market's Lowest-Cost S&P 500 ETF - T...,https://news.google.com/rss/articles/CBMiTWh0d...,"Wed, 30 Aug 2023 01:02:00 GMT",TipRanks
90,Prospect Capital reports Q4 results (NASDAQ:PS...,https://news.google.com/rss/articles/CBMiSWh0d...,"Tue, 29 Aug 2023 20:08:01 GMT",Seeking Alpha


In [10]:
df['Final_URL'] = ''

In [11]:
def beautifulsoup_extract_text_fallback(response_content):
    
    # Create the beautifulsoup object:
    soup = BeautifulSoup(response_content, 'html.parser')
    
    # Finding the text:
    text = soup.find_all(text=True)
    
    # Remove unwanted tag elements:
    cleaned_text = ''
    blacklist = [
        '[document]',
        'noscript',
        'header',
        'html',
        'meta',
        'head', 
        'input',
        'script',
        'style',]

    # Then we will loop over every item in the extract text and make sure that the beautifulsoup4 tag
    # is NOT in the blacklist
    for item in text:
        if item.parent.name not in blacklist:
            cleaned_text += '{} '.format(item)
            
    # Remove any tab separation and strip the text:
    cleaned_text = cleaned_text.replace('\t', '')
    return cleaned_text.strip()

def extract_text_from_single_web_page(url):
    downloaded_url = trafilatura.fetch_url(url)
    try:
        a = trafilatura.extract(downloaded_url, output_format="json", with_metadata=True, include_comments=False, date_extraction_params={'extensive_search': True, 'original_date': True})
    except AttributeError:
        a = trafilatura.extract(downloaded_url, output_format="json", with_metadata=True, date_extraction_params={'extensive_search': True, 'original_date': True})
    if a:
        json_output = json.loads(a)
        return json_output['text']
    else:
        try:
            resp = requests.get(url)
            if resp.status_code == 200:
                return beautifulsoup_extract_text_fallback(resp.content)
            else:
                return np.nan
        except MissingSchema:
            return np.nan

In [12]:


# Loop through the DataFrame rows
for i in range(len(df)):
    single_url = df['url'][i]
    text = extract_text_from_single_web_page(url=single_url)

    # Regular expression pattern to match URLs
    url_pattern = r'https?://[^\s/$.?#].[^\s]*'

    # Find all matches of the URL pattern in the text
    urls = re.findall(url_pattern, text)

    # Store the first extracted URL (if available) in the 'Final_URL' column
    if urls:
        df.at[i, 'Final_URL'] = urls[0]
    else:
        df.at[i, 'Final_URL'] = None  # or any other default value

# Print the DataFrame to see the updated 'Final_URL' column
df


,title,url,published_date,publisher,Final_URL
0,"Apple (NASDAQ:AAPL) Gains Slightly, Though Har...",https://news.google.com/rss/articles/CBMiWGh0d...,"Wed, 30 Aug 2023 16:29:57 GMT",TipRanks,https://www.tipranks.com/news/apple-nasdaqaapl...
1,Unusual Put Option Trade in Apple (AAPL) Worth...,https://news.google.com/rss/articles/CBMiVmh0d...,"Tue, 29 Aug 2023 15:27:00 GMT",Nasdaq,https://www.nasdaq.com/articles/unusual-put-op...
2,Apple Inc (AAPL) Stock Gains 1.82% This Week: ...,https://news.google.com/rss/articles/CBMib2h0d...,"Thu, 24 Aug 2023 19:01:41 GMT",InvestorsObserver,https://www.investorsobserver.com/news/stock-u...
3,"After Hours Most Active for Aug 25, 2023 : AAP...",https://news.google.com/rss/articles/CBMiemh0d...,"Fri, 25 Aug 2023 20:29:00 GMT",Nasdaq,https://www.nasdaq.com/articles/after-hours-mo...
4,"After Hours Most Active for Aug 28, 2023 : IGS...",https://news.google.com/rss/articles/CBMieWh0d...,"Mon, 28 Aug 2023 20:29:00 GMT",Nasdaq,https://www.nasdaq.com/articles/after-hours-mo...
...,...,...,...,...,...
87,"Dow, Nasdaq Shed Triple Digits; VIX Snaps Losi...",https://news.google.com/rss/articles/CBMiVWh0d...,"Thu, 24 Aug 2023 20:27:00 GMT",Nasdaq,https://www.nasdaq.com/articles/dow-nasdaq-she...
88,Warren Buffett Will Collect $4.31 Billion in A...,https://news.google.com/rss/articles/CBMid2h0d...,"Fri, 25 Aug 2023 09:06:00 GMT",Nasdaq,https://www.nasdaq.com/articles/warren-buffett...
89,SPLG: The Market's Lowest-Cost S&P 500 ETF - T...,https://news.google.com/rss/articles/CBMiTWh0d...,"Wed, 30 Aug 2023 01:02:00 GMT",TipRanks,https://www.tipranks.com/news/article/splg-the...
90,Prospect Capital reports Q4 results (NASDAQ:PS...,https://news.google.com/rss/articles/CBMiSWh0d...,"Tue, 29 Aug 2023 20:08:01 GMT",Seeking Alpha,https://seekingalpha.com/news/4007254-prospect...


In [13]:
df['Description'] = ''

In [14]:
# Function to extract text using BeautifulSoup
def beautifulsoup_extract_text_fallback(response_content):
        # Create the BeautifulSoup object:
    soup = BeautifulSoup(response_content, 'html.parser')
    
    # Finding the text:
    text = soup.find_all(text=True)
    
    # Remove unwanted tag elements:
    cleaned_text = ''
    blacklist = [
        '[document]',
        'noscript',
        'header',
        'html',
        'meta',
        'head', 
        'input',
        'script',
        'style',
    ]

    # Loop over every item in the extracted text and make sure that the BeautifulSoup4 tag
    # is NOT in the blacklist
    for item in text:
        if item.parent.name not in blacklist:
            cleaned_text += '{} '.format(item)
            
    # Remove any tab separation and strip the text:
    cleaned_text = cleaned_text.replace('\t', '')
    return cleaned_text.strip()

# Function to extract text from a single web page
def extract_text_from_single_web_page(url):
    resp = None  # Initialize resp variable
    try:
        resp = requests.get(url, timeout=30)  # Set timeout to 30 seconds
        resp.raise_for_status()  # Raise an exception if the status code is not 200
        downloaded_url = trafilatura.fetch_url(url)
        try:
            a = trafilatura.extract(downloaded_url, output_format="json", with_metadata=True, include_comments=False, date_extraction_params={'extensive_search': True, 'original_date': True})
        except AttributeError:
            a = trafilatura.extract(downloaded_url, output_format="json", with_metadata=True, date_extraction_params={'extensive_search': True, 'original_date': True})
        if a:
            json_output = json.loads(a)
            return json_output['text']
        else:
            return beautifulsoup_extract_text_fallback(resp.content)
    except (requests.exceptions.RequestException, requests.exceptions.HTTPError):
        if resp is not None:
            return " ".join(beautifulsoup_extract_text_fallback(resp.content).split()[:10])
        else:
            return np.nan

# Iterate over URLs and extract text
for i in range(len(df)):
    url = df['Final_URL'][i]
    text = extract_text_from_single_web_page(url=url)
    df['Description'][i] = text


C:\Users\ManojKumar\AppData\Local\Temp\ipykernel_25044\313010365.py:7: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  text = soup.find_all(text=True)


In [59]:
for i in range(len(df)):
    if pd.isna(df['Description'][i]):
        df['Description'][i] = df['title'][i]

In [60]:
df

,title,url,published_date,publisher,Final_URL,Description
0,Apple Stock Dividend Analysis: August 2023 Edi...,https://news.google.com/rss/articles/CBMiUGh0d...,"Wed, 23 Aug 2023 15:53:38 GMT",Forbes,https://www.forbes.com/sites/investor-hub/arti...,"In December 2018, you could buy a share of App..."
1,Best Dividend Stocks: Apple Stock vs. Microsof...,https://news.google.com/rss/articles/CBMiVWh0d...,"Mon, 21 Aug 2023 13:42:00 GMT",Nasdaq,https://www.nasdaq.com/articles/best-dividend-...,Best Dividend Stocks: Apple Stock vs. Microsof...
2,3 Dividend Stocks With High Returns On Investe...,https://news.google.com/rss/articles/CBMiZGh0d...,"Thu, 24 Aug 2023 19:33:00 GMT",Barchart,https://www.barchart.com/story/news/19680805/3...,ERROR: The request could not be satisfied 403 ...
3,Premarket Mover: Apple Inc (AAPL) Up 0.53% - I...,https://news.google.com/rss/articles/CBMiWmh0d...,"Wed, 23 Aug 2023 13:18:35 GMT",InvestorsObserver,https://www.investorsobserver.com/news/stock-u...,"Apple Inc (AAPL) is up Wednesday morning, with..."
4,Unusual Put Option Trade in Apple (AAPL) Worth...,https://news.google.com/rss/articles/CBMiVmh0d...,"Thu, 24 Aug 2023 15:07:00 GMT",Nasdaq,https://www.nasdaq.com/articles/unusual-put-op...,Unusual Put Option Trade in Apple (AAPL) Worth...
...,...,...,...,...,...,...
89,"Analyst Issues Dire Bitcoin Warning, Says BTC ...",https://news.google.com/rss/articles/CBMigAFod...,"Thu, 24 Aug 2023 09:15:31 GMT",The Daily Hodl,https://dailyhodl.com/2023/08/24/analyst-issue...,Crypto analyst Justin Bennett is warning that ...
90,3 Wealthsimple Stocks To Buy Right Now - The M...,https://news.google.com/rss/articles/CBMiRmh0d...,"Mon, 21 Aug 2023 16:00:00 GMT",The Motley Fool Canada,https://www.fool.ca/2023/08/21/3-wealthsimple-...,403 Forbidden 403 Forbidden nginx
91,"The Zacks Analyst Blog Highlights Apple, NVIDI...",https://news.google.com/rss/articles/CBMicGh0d...,"Mon, 21 Aug 2023 13:04:00 GMT",Nasdaq,https://www.nasdaq.com/articles/the-zacks-anal...,"The Zacks Analyst Blog Highlights Apple, NVIDI..."
92,If You Invested $1000 In Apple Stock When Appl...,https://news.google.com/rss/articles/CBMioQFod...,"Wed, 23 Aug 2023 07:33:00 GMT",Investing.com UK,https://uk.investing.com/news/stock-market-new...,Attention Required! | Cloudflare Please enable...


,title,url,published_date,publisher
0,Apple Stock Dividend Analysis: August 2023 Edi...,https://news.google.com/rss/articles/CBMiUGh0d...,"Wed, 23 Aug 2023 15:53:38 GMT",Forbes
1,Unusual Put Option Trade in Apple (AAPL) Worth...,https://news.google.com/rss/articles/CBMiVmh0d...,"Thu, 24 Aug 2023 15:07:00 GMT",Nasdaq
2,Premarket Mover: Apple Inc (AAPL) Up 0.53% - I...,https://news.google.com/rss/articles/CBMiWmh0d...,"Wed, 23 Aug 2023 13:18:35 GMT",InvestorsObserver
3,Apple Stock Forecast: Can $170 Reverse The Tre...,https://news.google.com/rss/articles/CBMiZmh0d...,"Tue, 22 Aug 2023 22:34:00 GMT",The Coin Republic
4,Apple (NASDAQ:AAPL) Deemed 'Most Under-Owned' ...,https://news.google.com/rss/articles/CBMiTWh0d...,"Wed, 23 Aug 2023 14:44:26 GMT",TipRanks
5,It's Time to Load Up on Apple Stock - The Motl...,https://news.google.com/rss/articles/CBMiTWh0d...,"Mon, 21 Aug 2023 11:16:00 GMT",The Motley Fool
6,Apple Stock Price Prediction 2023: Is AAPL Tur...,https://news.google.com/rss/articles/CBMiZWh0d...,"Sat, 19 Aug 2023 18:32:00 GMT",The Coin Republic
7,Apple Inc. (AAPL) is Attracting Investor Atten...,https://news.google.com/rss/articles/CBMicGh0d...,"Mon, 21 Aug 2023 13:00:00 GMT",Nasdaq
8,Riverview Capital Advisers LLC Sells 1607 Shar...,https://news.google.com/rss/articles/CBMiTGh0d...,"Fri, 25 Aug 2023 08:18:20 GMT",MarketBeat
9,Apple's Buyback Blunder: 3 Reasons It's Not a ...,https://news.google.com/rss/articles/CBMiZ2h0d...,"Tue, 22 Aug 2023 15:15:33 GMT",InvestorPlace


08/25/2023 03:52:25 PM - error in sitename extraction: string index out of range
08/25/2023 03:52:31 PM - error in sitename extraction: string index out of range


,title,url,published_date,publisher,Final_URL,Description
0,Apple Stock Dividend Analysis: August 2023 Edi...,https://news.google.com/rss/articles/CBMiUGh0d...,"Wed, 23 Aug 2023 15:53:38 GMT",Forbes,https://www.forbes.com/sites/investor-hub/arti...,"In December 2018, you could buy a share of App..."
1,Unusual Put Option Trade in Apple (AAPL) Worth...,https://news.google.com/rss/articles/CBMiVmh0d...,"Thu, 24 Aug 2023 15:07:00 GMT",Nasdaq,https://www.nasdaq.com/articles/unusual-put-op...,Unusual Put Option Trade in Apple (AAPL) Worth...
2,Premarket Mover: Apple Inc (AAPL) Up 0.53% - I...,https://news.google.com/rss/articles/CBMiWmh0d...,"Wed, 23 Aug 2023 13:18:35 GMT",InvestorsObserver,https://www.investorsobserver.com/news/stock-u...,"Apple Inc (AAPL) is up Wednesday morning, with..."
3,Apple Stock Forecast: Can $170 Reverse The Tre...,https://news.google.com/rss/articles/CBMiZmh0d...,"Tue, 22 Aug 2023 22:34:00 GMT",The Coin Republic,https://www.thecoinrepublic.com/2023/08/22/app...,"- 1 Apple stock is trying to sustain, CMP is $..."
4,Apple (NASDAQ:AAPL) Deemed 'Most Under-Owned' ...,https://news.google.com/rss/articles/CBMiTWh0d...,"Wed, 23 Aug 2023 14:44:26 GMT",TipRanks,https://www.tipranks.com/news/apple-deemed-mos...,Error Page Security Violation (503) www.tipran...
5,It's Time to Load Up on Apple Stock - The Motl...,https://news.google.com/rss/articles/CBMiTWh0d...,"Mon, 21 Aug 2023 11:16:00 GMT",The Motley Fool,https://www.fool.com/investing/2023/08/21/its-...,Following the tech-heavy Nasdaq's pullback las...
6,Apple Stock Price Prediction 2023: Is AAPL Tur...,https://news.google.com/rss/articles/CBMiZWh0d...,"Sat, 19 Aug 2023 18:32:00 GMT",The Coin Republic,https://www.thecoinrepublic.com/2023/08/19/app...,- 1 Apple’s stock price decreased by 1.60% ove...
7,Apple Inc. (AAPL) is Attracting Investor Atten...,https://news.google.com/rss/articles/CBMicGh0d...,"Mon, 21 Aug 2023 13:00:00 GMT",Nasdaq,https://www.nasdaq.com/articles/apple-inc.-aap...,Apple Inc. (AAPL) is Attracting Investor Atten...
8,Riverview Capital Advisers LLC Sells 1607 Shar...,https://news.google.com/rss/articles/CBMiTGh0d...,"Fri, 25 Aug 2023 08:18:20 GMT",MarketBeat,https://www.marketbeat.com/instant-alerts/nasd...,Riverview Capital Advisers LLC cut its holding...
9,Apple's Buyback Blunder: 3 Reasons It's Not a ...,https://news.google.com/rss/articles/CBMiZ2h0d...,"Tue, 22 Aug 2023 15:15:33 GMT",InvestorPlace,https://investorplace.com/2023/08/apples-buyba...,If there’s one thing that feels like Christmas...
